In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("gpt2")

2024-03-08 12:43:13.041242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name '_LazyModule' from 'transformers.utils' (/opt/conda/lib/python3.10/site-packages/transformers/utils/__init__.py)

In [ ]:
model.config

In [3]:
prompt = "\n"

In [4]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [5]:
from torch.utils.data import DataLoader
from datasets import load_dataset
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1")
train_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="test")
                

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
batch_size = 12
#test_dataset_text = [test_dataset[i : i + batch_size]["text"] for i in range(0, len(test_dataset), batch_size)]
#def batch_iterator():
#    for i in range(0, len(dataset), batch_size):
#        yield dataset[i : i + batch_size]["text"]

In [7]:
# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
embedding_size

50257

In [6]:
column_names = dataset["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)
block_size = tokenizer.model_max_length

Running tokenizer on dataset:   0%|          | 0/4358 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/36718 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [9]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    desc=f"Grouping texts in chunks of {block_size}",
)

train_dataset = lm_datasets["train"]
print(len(train_dataset))
eval_dataset = lm_datasets["validation"]
print(len(eval_dataset))
test_dataset = lm_datasets["test"]
print(len(test_dataset))

2318
240
274


In [10]:
import random
for index in random.sample(range(len(eval_dataset)), 1):
    print(tokenizer.decode(eval_dataset[index]['input_ids'])[:100])
    print(tokenizer.decode(eval_dataset[index]['labels'])[:100])
    #print(f"Sample {index} of the training set: {train_dataset[index]}.")

 due east from Gould City to Naubinway and then along the lake to Epoufette. The former route throug
 due east from Gould City to Naubinway and then along the lake to Epoufette. The former route throug


In [9]:
# DataLoaders creation:
from transformers import default_data_collator

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=batch_size
)
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=batch_size
)
print(len(train_dataloader))
print(len(eval_dataloader))
print(len(test_dataloader))

3060
314
364


In [22]:
import torch
for epoch in range(0, 1):
    active_dataloader = eval_dataloader
    model.eval()
    losses = []
    for step, batch in enumerate(active_dataloader):
        print(step)
        print(batch)
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))
        print(loss)

    losses = torch.cat(losses)
    try:
        eval_loss = torch.mean(losses)
        perplexity = math.exp(eval_loss)
    except OverflowError:
        perplexity = float("inf")

    logger.info(f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}")

    print({
            "perplexity": perplexity,
            "eval_loss": eval_loss,
            "epoch": epoch,
            "step": completed_steps,
        },
        step=completed_steps,
    )

NameError: name 'model' is not defined

In [13]:
import math

In [17]:
try:
    eval_loss = torch.mean(losses)
    perplexity = math.exp(eval_loss)
except OverflowError:
    perplexity = float("inf")

print((f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}"))

print({
        "perplexity": perplexity,
        "eval_loss": eval_loss,
        "epoch": epoch,
    }
)

epoch 0: perplexity: 30.63013824217404 eval_loss: 3.4219844341278076
{'perplexity': 30.63013824217404, 'eval_loss': tensor(3.4220), 'epoch': 0}


In [21]:
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    LlamaConfig,
    default_data_collator,
)
  
# Create DataLoaders for the training and validation dataset
train_dataloader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=train_config.batch_size_training,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    sampler=train_sampler if train_sampler else None,
    drop_last=True,
    collate_fn=default_data_collator,
)

for i, batch in enumerate(train_dataloader):
    print(batch)
    break

ImportError: cannot import name 'LlamaForCausalLM' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [32]:
tokens, text = (tokenized_datasets["train"]["input_ids"][10], train_dataset["text"][10])
text

' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a charac

In [31]:
from transformers import GPT2LMHeadModel
model_lmhead = GPT2LMHeadModel.from_pretrained("gpt2")


In [46]:
def measure_accuracy(inputs, labels):
    _, accuracy =  inputs.max(dim=-1).indices.eq(labels).unique(return_counts=True)
    #if length is one, then accuracy only contains false values
    accuracy = accuracy[1] / (accuracy[0] + accuracy[1]) * 100 if len(accuracy) == 2 else 0.0
    return accuracy

In [42]:
len(tokens)

324

In [35]:
out = model_lmhead(torch.Tensor(tokens).to(dtype=torch.long))

In [44]:
probs = torch.nn.functional.softmax(out.logits, dim=-1)

tensor([[1.3758e-04, 3.6045e-04, 1.8377e-05,  ..., 2.4572e-06, 1.0243e-06,
         6.1406e-04],
        [7.2173e-05, 2.8046e-05, 9.7994e-07,  ..., 4.2035e-08, 1.1729e-07,
         1.3800e-05],
        [4.1298e-05, 2.8809e-04, 1.3808e-04,  ..., 2.2889e-08, 5.0886e-08,
         6.8176e-05],
        ...,
        [6.7620e-06, 1.0283e-05, 2.5769e-06,  ..., 1.5717e-09, 8.4749e-11,
         3.3997e-03],
        [1.1863e-04, 3.0943e-04, 2.7778e-05,  ..., 2.4461e-08, 2.7869e-08,
         6.0270e-03],
        [1.8355e-05, 7.0140e-04, 9.1146e-05,  ..., 5.1784e-09, 1.4598e-09,
         6.4578e-03]], grad_fn=<SoftmaxBackward0>)

In [53]:
from tiktoken import get_encoding
tokenizer = get_encoding("gpt2")
gen_text = tokenizer.decode(probs.max(dim=-1).indices.tolist())

## generated text looks nothing like input text, i think i am using the forward pass incorrectly

In [54]:
gen_text

'\n isThe story system is which gameastersksron, , and a over from from theiscria\'s .\n the, the can a character from a unique- andlevel- arrow. the battlefield..\n selected unit is selected, the unit can to unit to the battlefield using a person-@ order view\n character can be be in per mission-@ position. and can can be moved a @ of a same of a characters.s.\n character has a unique of a to 1 that to the own Pointsge . The to three characters can be selected to a single field at Each a , the can be upon to they is to them or and as a character drops or ), ) or depleted or their hit down of a attacks .\n character has a actionsspecialions " that which that to each character , Each can: into three Pot " " , " is used abilities that can activealtered for changed specified by the player . the be be or hinder the character\'s or " Special Potentialentials " , which are abilities- the game . are have aons to characters character . Each gain more Potentials , players character must a speci

In [47]:
acc = measure_accuracy(probs, torch.Tensor(tokens).to(dtype=torch.long))

In [48]:
acc

tensor(0.3086)